# Profiling Test

In [52]:
%load_ext autoreload
%autoreload 2

from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

# default processer
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384. You can set min_pixels and max_pixels according to your needs, such as a token count range of 256-1280, to balance speed and memory usage.
min_pixels = 256*28*28
max_pixels = 1280*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
model.print_time()
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\colin\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\utils\modeling.py:1390: UserWarning: Current model requires 469765672 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00,  5.25it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.
c:\Users\colin\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1902: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generat

2
torch.Size([4988, 1176])
tensor([[ 1, 58, 86]], dtype=torch.int32)
PatchEmbed 3, 2, 14
torch.Size([4988, 40])
torch.Size([4988, 1280])
16


: 

# Baseline Evaluation

In [2]:
%load_ext autoreload
%autoreload 2

import torch
from transformers import Qwen2VLConfig, Qwen2VLForConditionalGeneration, TrainingArguments, Trainer, AutoProcessor
import evaluate
import numpy as np
from datasets import load_dataset
import time

import baseline
import efficient
from utils import get_dataset, train

c:\Users\colin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Make dataset

In [90]:
min_pixels = 16*28*28
max_pixels = 80*28*28
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

messages = [{
    "role": "user",
    "content": [
        {"type": "image"},
    ],
}]

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

datasets = load_dataset("mnist", cache_dir='.cache')
train_dataset = datasets["train"].select(range(1000))
test_dataset = datasets["test"].select(range(100))

def process_function(datum):
    return processor(text=text, images=datum["image"], videos=None, padding=True, return_tensors="pt")

train_dataset = train_dataset.map(process_function, batched=False).shuffle(seed=1512)
test_dataset = test_dataset.map(process_function, batched=False).shuffle(seed=1512)

Map: 100%|██████████| 100/100 [00:00<00:00, 709.09 examples/s]


Train and evaluate baseline model

In [57]:
baseline_config = Qwen2VLConfig(
    hidden_size=16,
    intermediate_size=128,
    num_hidden_layers=3,
    num_attention_heads=8,
    rope_scaling={"mrope_section": [1, 1, 2], "type": "mrope"},
    vision_config={"hidden_size": 64,
                   "in_channels": 3,
                   "embed_dim": 64,
                   "num_heads": 1,
                   "spatial_merge_size": 2,
                   "patch_size": 14,
                   "temporal_patch_size": 2,
                   "depth": 3,
                   "vocab_size": 10,
                   "spatial_merge_size": 2},
    _name_or_path="baseline",
    image_token_id=151655,
)

#print(baseline_config)

baseline_model = baseline.Qwen2VLBaseline(baseline_config.vision_config)

In [8]:
print(train_dataset)

Dataset({
    features: ['image', 'label', 'input_ids', 'attention_mask', 'pixel_values', 'image_grid_thw'],
    num_rows: 1
})


In [ ]:
class Args:
    device = "cpu"
    lr_net = 0.005
    epoch_eval_train = 10

args = Args()

In [59]:
train(0, baseline_model, train_dataset, test_dataset, args)

[2024-12-03 17:02:47] Evaluate_00: epoch = 0003 train time = 24 s train loss = 1.056103 train acc = 0.6800, test acc = 0.5000


Train and evaluate efficient model

In [88]:
efficient_config = Qwen2VLConfig(
    hidden_size=16,
    intermediate_size=128,
    num_hidden_layers=3,
    num_attention_heads=8,
    rope_scaling={"mrope_section": [1, 1, 2], "type": "mrope"},
    vision_config={"hidden_size": 64,
                   "in_channels": 3,
                   "embed_dim": 64,
                   "num_heads": 1,
                   "spatial_merge_size": 2,
                   "patch_size": 14,
                   "temporal_patch_size": 2,
                   "depth": 3,
                   "vocab_size": 10,
                   "spatial_merge_size": 2},
    _name_or_path="efficient",
    image_token_id=151655,
)

efficient_model = efficient.Qwen2VLEfficient(efficient_config.vision_config)

In [91]:
class Args:
    device = "cuda"
    lr_net = 0.005
    epoch_eval_train = 10

args = Args()

In [92]:
train(0, efficient_model, train_dataset, test_dataset, args)

[2024-12-03 17:50:08] Evaluate_00: epoch = 0010 train time = 316 s train loss = 0.037296 train acc = 0.9920, test acc = 0.8900


In [37]:
x = torch.randn([64,64]).numpy()
np.sum(np.abs(x), 0)

array([56.920467, 47.16939 , 56.242165, 56.261654, 53.352417, 49.153526,
       49.006687, 47.99332 , 47.67888 , 51.1504  , 46.553295, 59.930485,
       50.166054, 53.989044, 45.1649  , 54.88476 , 45.1556  , 49.01019 ,
       56.02619 , 53.802086, 48.837364, 50.971104, 50.13207 , 60.619606,
       53.63239 , 49.137062, 57.631992, 47.799107, 50.219166, 56.934956,
       51.861347, 53.022583, 48.632187, 51.89994 , 53.112076, 51.023262,
       46.39769 , 52.821156, 51.804142, 51.369476, 58.84605 , 52.86758 ,
       62.422626, 54.64633 , 55.126945, 57.355015, 54.241074, 54.07905 ,
       41.357086, 53.124294, 46.557323, 53.85053 , 41.190422, 45.550182,
       51.022633, 53.425903, 58.612766, 39.902985, 62.082855, 54.20802 ,
       55.088013, 55.21963 , 46.374645, 51.68664 ], dtype=float32)

In [36]:
most_impactful = np.argpartition(np.sum(np.abs(x), 0), 32)[32:]
x[most_impactful]

array([[-0.38639838, -0.98506033,  1.0919193 , ...,  0.45764935,
         0.34476206, -0.77541775],
       [ 1.2277838 ,  0.76195467,  1.4347185 , ..., -0.48140383,
        -0.890773  ,  0.98833424],
       [-0.13808826,  1.6146613 , -0.7909208 , ...,  0.6706576 ,
        -0.6683063 ,  0.45490786],
       ...,
       [ 1.2200717 ,  0.9622775 , -1.6472394 , ..., -0.49833935,
        -0.43424693,  0.24749589],
       [-0.79002994,  0.05223804, -0.7194966 , ...,  0.881318  ,
         1.3307374 ,  0.07237424],
       [ 0.96025515, -1.0600721 , -1.0369935 , ...,  1.2501028 ,
         1.300068  ,  0.79003644]], dtype=float32)